In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn import datasets
from sklearn.cluster import AgglomerativeClustering, DBSCAN

n_samples = 1500
dataset = datasets.make_blobs(n_samples=n_samples, centers=2, center_box=(-7.0, 7.5),
                              cluster_std=[1.4, 1.7],
                              random_state=42)
X_2, _ = datasets.make_blobs(n_samples=n_samples, random_state=170, centers=[[-4, -3]], cluster_std=[1.9])
transformation = [[1.2, -0.8], [-0.4, 1.7]]
X_2 = np.dot(X_2, transformation)
X, y = np.concatenate((dataset[0], X_2)), np.concatenate((dataset[1], np.array([2] * len(X_2))))

Коэффициент силуэта можно посчитать при помощи реализации из библиотеки sklearn:

In [2]:
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

# сначала получим предсказанные кластеры при помощи метода кластеризации
kmeans = KMeans(n_clusters=3, random_state=42)
kmeans.fit(X)
kmeans_pred = kmeans.labels_

# теперь посчитаем коэффициент силуэта
silhouette_score(X=X, labels=kmeans_pred, metric='euclidean')

0.5131209788437305

В качестве параметров в функции silhouette_score используются:

X — массив признаков объектов выборки или массив попарных расстояний между объектами;
Y — массив предсказанных кластеров для объектов выборки;
metric — метрика, используемая для вычисления расстояния между объектами, мы будем использовать euclidean (Евклидово расстояние)

Задание 7.11.1

Обучите модель GaussianMixture с параметрами n_components=3 и random_state=42 на признаках исходного датасета. Посчитайте коэффициент силуэта для получившейся кластеризации. Ответ округлите до десятых и запишите с точкой.

In [3]:
from sklearn.mixture import GaussianMixture

In [4]:
em = GaussianMixture(n_components=3, random_state=42)

In [5]:
em.fit(X)

GaussianMixture(n_components=3, random_state=42)

In [6]:
labels = em.predict(X)

In [7]:
# Считаем коэфф. силуэта
silhouette_score(X=X, labels=labels, metric='euclidean')

0.49893287606943293

Задание 7.11.2

Сравните результаты кластеризации четырёх рассмотренных алгоритмов на исходном датасете при помощи коэффициента силуэта, инициализируйте алгоритмы со следующими параметрами:

K-means — n_clusters=3, random_state=42
EM-алгоритм (GaussianMixture) — n_components=3, random_state=42
Агломеративная кластеризация – n_clusters=3
DBSCAN – eps=0.9, min_samples=35
Укажите максимальное значение коэффициента силуэта, полученное при помощи данных моделей. Ответ округлите до сотых и запишите с точкой.

In [8]:
kmean = KMeans(n_clusters=3, random_state=42)
em = GaussianMixture(n_components=3, random_state=42)
agglomerative = AgglomerativeClustering(n_clusters=3)
dbscan = DBSCAN(eps=0.9 ,min_samples=35)

In [9]:
kmean.fit(X)
em.fit(X)
agglomerative.fit(X)
dbscan.fit(X)

DBSCAN(eps=0.9, min_samples=35)

In [10]:
lebels_kmean = kmean.labels_
lebels_em = em.predict(X)
lebels_agg = agglomerative.labels_
lebels_dbscan = dbscan.labels_

In [11]:
print('KMean silhouette_score',round(silhouette_score(X=X, labels=lebels_kmean, metric='euclidean'),2))
print('EM silhouette_score',round(silhouette_score(X=X, labels=lebels_em, metric='euclidean'),2))
print('Agglomerative silhouette_score',round(silhouette_score(X=X, labels=lebels_agg, metric='euclidean'),2))
print('DBSCAN silhouette_score',round(silhouette_score(X=X, labels=lebels_dbscan, metric='euclidean'),2))

KMean silhouette_score 0.51
EM silhouette_score 0.5
Agglomerative silhouette_score 0.48
DBSCAN silhouette_score 0.45


Задание 7.11.3

Подберите оптимальное количество кластеров с помощью коэффициента силуэта. Для этого найдите такое число кластеров, при котором значение коэффициента будет максимальным.

В трёх из рассмотренных нами алгоритмов необходимо задать число кластеров при инициализации: K-means, EM-алгоритм и агломеративная кластеризация.

Найдите значение коэффициента силуэта для данных алгоритмов при числе кластеров от 2 до 10 включительно. Для K-means и EM-алгоритма установите значение random_state=42.

В качестве ответа через пробел введите число кластеров, при котором значение коэффициента силуэта для результатов кластеризации было наибольшим для каждого из алгоритмов. Вводите в следующем порядке: K-means, EM-алгоритм, агломеративная кластеризация.

In [12]:
silhouette_kmeans = dict()
silhouette_em = dict()
silhouette_agg = dict()
for cluster in range(2,11,1):
    
    kmean = KMeans(n_clusters=cluster, random_state=42)
    em = GaussianMixture(n_components=cluster, random_state=42)
    agglomerative = AgglomerativeClustering(n_clusters=cluster)

    kmean.fit(X)
    em.fit(X)
    agglomerative.fit(X)

    lebels_kmean = kmean.labels_
    lebels_em = em.predict(X)
    lebels_agg = agglomerative.labels_
    
    silhouette_kmeans.update({cluster: round(silhouette_score(X=X, labels=lebels_kmean, metric='euclidean'),3)})
    silhouette_em.update({cluster: round(silhouette_score(X=X, labels=lebels_em, metric='euclidean'),3)})
    silhouette_agg.update({cluster: round(silhouette_score(X=X, labels=lebels_agg, metric='euclidean'),3)})



In [13]:
print('Кластер с наибольшим коэффициентом силуэта KMeans',max(silhouette_kmeans, key=lambda k: silhouette_kmeans[k]))
print('Кластер с наибольшим коэффициентом силуэта EM',max(silhouette_em, key=lambda k: silhouette_em[k]))
print('Кластер с наибольшим коэффициентом силуэта Agglomerative',max(silhouette_agg, key=lambda k: silhouette_agg[k]))

Кластер с наибольшим коэффициентом силуэта KMeans 3
Кластер с наибольшим коэффициентом силуэта EM 4
Кластер с наибольшим коэффициентом силуэта Agglomerative 4


# Однородность. Практика

In [14]:
from sklearn.cluster import KMeans
from sklearn.metrics.cluster import homogeneity_score
from sklearn.preprocessing import StandardScaler


# сначала получим предсказанные кластеры при помощи метода кластеризации
kmeans = KMeans(n_clusters=3, random_state=42)
X = StandardScaler().fit_transform(X)
kmeans.fit(X); kmeans_pred = kmeans.labels_ # теперь посчитаем однородность homogeneity_score(labels_true=y, labels_pred=kmeans_pred)

Задание 7.12.1

Сравните результаты кластеризации алгоритмов k-means, GaussianMixture, AgglomerativeClustering и DBSCAN на исходном датасете при помощи однородности, инициализируйте алгоритмы со следующими параметрами:

k-means — n_clusters=3, random_state=42
GaussianMixture — n_components=3, random_state=42
AgglomerativeClustering — n_clusters=3
DBSCAN — eps=0.9, min_samples=35
В качестве ответа укажите максимальное значение однородности, полученное при помощи данных моделей. Ответ округлите до сотых и запишите с точкой.

При решении задания модуля стандартизация должна быть включена.

In [15]:
kmeans = KMeans(n_clusters=3, random_state=42)
em = GaussianMixture(n_components=3, random_state=42)
agg = AgglomerativeClustering(n_clusters=3)
dbscan = DBSCAN(eps=0.9, min_samples=35)

In [16]:
kmeans.fit(X)
em.fit(X)
agg.fit(X)
dbscan.fit(X)


DBSCAN(eps=0.9, min_samples=35)

In [17]:
labels_k = kmeans.labels_
labels_em = em.predict(X)
labels_agg = agg.labels_
labels_dbscan = dbscan.labels_


In [18]:
print('Полнота KMean',round(homogeneity_score(y, labels_k), 2))
print('Полнота EM',round(homogeneity_score(y, labels_em), 2))
print('Полнота Agglomerative',round(homogeneity_score(y, labels_agg), 2))
print('Полнота DBSCAN',round(homogeneity_score(y, labels_dbscan),2 ))

Полнота KMean 0.8
Полнота EM 0.93
Полнота Agglomerative 0.91
Полнота DBSCAN 0.0


# Полнота. Практика

In [19]:
from sklearn.cluster import KMeans
from sklearn.metrics.cluster import completeness_score

# сначала получим предсказанные кластеры при помощи метода кластеризации
kmeans = KMeans(n_clusters=3, random_state=42)
kmeans.fit(X)
kmeans_pred = kmeans.labels_

# теперь посчитаем полноту
completeness_score(labels_true=y, labels_pred=kmeans_pred)

0.7808712092278982

Задание 7.13.1

Обучите модель GaussianMixture с параметрами n_components = 3 и random_state = 42 на признаках исходного датасета. Посчитайте полноту для получившейся кластеризации. Ответ округлите до сотых и запишите с точкой. Не забудьте о стандартизации, она должна быть включена.

In [20]:
em = GaussianMixture(n_components=3, random_state=42)
em.fit(X)
labels_agg = em.predict(X)
round(completeness_score(y, labels_agg), 2)

0.93

Задание 7.13.2

Сравните результаты кластеризации алгоритмов k-means, GaussianMixture, AgglomerativeClustering и DBSCAN на исходном датасете при помощи полноты, инициализируйте алгоритмы со следующими параметрами:

k-means — n_clusters=3, random_state=42
GaussianMixture — n_components=3, random_state=42
AgglomerativeClustering — n_clusters=3
DBSCAN — eps=0.9, min_samples=35
В качестве ответа укажите максимальное значение полноты, полученное при помощи данных моделей.

Подсказка: При решении задания модуля стандартизация должна быть включена. Ответ округлите до сотых и запишите с точкой.

In [21]:
kmean = KMeans(n_clusters=3, random_state=42)
em = GaussianMixture(n_components=3, random_state=42)
agg = AgglomerativeClustering(n_clusters=3)
dbscan = DBSCAN(eps=0.9, min_samples=35)

In [22]:
kmean.fit(X)
em.fit(X)
agg.fit(X)
dbscan.fit(X)

DBSCAN(eps=0.9, min_samples=35)

In [23]:
labels_k = kmean.labels_
labels_em = em. predict(X)
labels_agg = agg.labels_
labels_dbscan = dbscan.labels_

In [24]:
print('Полнота KMeans', round(completeness_score(y, labels_k),2))
print('Полнота EM',round(completeness_score(y, labels_em),2))
print('Полнота Agglomerative',round(completeness_score(y, labels_agg),2))
print('Полнота DBSCAN',round(completeness_score(y, labels_dbscan),2))

Полнота KMeans 0.78
Полнота EM 0.93
Полнота Agglomerative 0.91
Полнота DBSCAN 0.08


# V-мера. Практика

In [25]:
from sklearn.cluster import KMeans
from sklearn.metrics.cluster import v_measure_score

# сначала получим предсказанные кластеры при помощи метода кластеризации
kmeans = KMeans(n_clusters=3, random_state=42)
kmeans.fit(X)
kmeans_pred = kmeans.labels_

# теперь посчитаем полноту
v_measure_score(labels_true=y, labels_pred=kmeans_pred)

0.791546668267586

Задание 7.14.1
Обучите модель GaussianMixture с параметрами n_components=3 и random_state=42 на признаках исходного датасета. Посчитайте v-меру для получившейся кластеризации.

Подсказка: При решении задания модуля стандартизация должна быть включена. Ответ округлите до сотых и запишите с точкой.

In [26]:
em = GaussianMixture(n_components=3, random_state=42)
em.fit(X)
labels_em = em.predict(X)
print('V_measure',round(v_measure_score(y, labels_em),2))

V_measure 0.93


Задание 7.14.2
Сравните результаты кластеризации алгоритмов k-means, GaussianMixture, AgglomerativeClustering и DBSCAN на исходном датасете при помощи v-меры, инициализируйте алгоритмы со следующими параметрами:

k-means — n_clusters=3, random_state=42
GaussianMixture — n_components=3, random_state=42
AgglomerativeClustering — n_clusters=3
DBSCAN — eps=0.9, min_samples=35
В качестве ответа укажите максимальное значение v-меры, полученное при помощи данных моделей. Ответ округлите до сотых и запишите с точкой.

In [27]:
kmeans = KMeans(n_clusters=3, random_state=42)
em = GaussianMixture(n_components=3, random_state=42)
agg = AgglomerativeClustering(n_clusters=3)
dbscan = DBSCAN(eps=0.9, min_samples=35)

In [28]:
kmeans.fit(X)
em.fit(X)
agg.fit(X)
dbscan.fit(X)


DBSCAN(eps=0.9, min_samples=35)

In [29]:
labels_k = kmeans.labels_
labels_em = em.predict(X)
labels_agg = agg.labels_
labels_dbscan = dbscan.labels_

In [30]:
print('V_measure KMeans', round(v_measure_score(y, labels_k),2))
print('V_measure EM',round(v_measure_score(y, labels_em),2))
print('V_measure Agglomerative',round(v_measure_score(y, labels_agg),2))
print('V_measure DBSCAN',round(v_measure_score(y, labels_dbscan),2))

V_measure KMeans 0.79
V_measure EM 0.93
V_measure Agglomerative 0.91
V_measure DBSCAN 0.0


Задание 7.14.3
Сравним модификации K-means с использованием случайной инициализации центроид и с использованием алгоритма K-means++ для инициализации центроид.

Для этого обучим на исходном датасете 2 модели k-means со следующими параметрами:

n_clusters=3, init='k-means++', n_init=1, random_state=42
n_clusters=3, init='random', n_init=1, random_state=42
В качестве ответа укажите максимальное значение v-меры, полученное при помощи данных моделей.

Подсказка: При решении задания модуля стандартизация должна быть включена. Ответ округлите до сотых и запишите с точкой.

In [31]:
kmeans_plus = KMeans(n_clusters=3, init='k-means++',n_init=1, random_state=42)
kmeans_random = KMeans(n_clusters=3, init='random', n_init=1, random_state=42)

In [32]:
kmeans_plus.fit(X)
kmeans_random.fit(X)

KMeans(init='random', n_clusters=3, n_init=1, random_state=42)

In [33]:
labels_k_plus = kmeans_plus.labels_
labels_k_random = kmeans_random.labels_

In [34]:
print('V_measure_k++',round(v_measure_score(y, labels_k_plus),2))
print('V_measure_random',round(v_measure_score(y, labels_k_random),2))

V_measure_k++ 0.79
V_measure_random 0.79


Задание 7.14.4
Теперь сравним k-means с ещё одной модификацией — K-means mini batch. Воспользоваться реализацией K-means mini batch в библиотеке sklearn можно следующим образом:

In [35]:
from sklearn.cluster import MiniBatchKMeans

kmeans_mini_batch = MiniBatchKMeans(n_clusters=
3, random_state=
42)
kmeans_mini_batch.fit(X)
kmeans_mini_batch_pred = kmeans_mini_batch.labels_

C:\Users\vetak\anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:887: UserWarning: MiniBatchKMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can prevent it by setting batch_size >= 2048 or by setting the environment variable OMP_NUM_THREADS=1
  warnings.warn(


Механизм кластеризации версии K-means mini batch схож с оригинальной версией алгоритма. Обучите на исходном датасете две модели:

k-means с параметрами n_clusters=3, n_init=1, random_state=42
MiniBatchKMeans с параметрами n_clusters=3, n_init=1, random_state=42
В качестве ответа введите максимальное значение v-меры, полученное при помощи данных моделей. В задании может понадобиться, а может не понадобиться нормализация и это нужно проверить во время решения задания. Ответ округлите до десятых и запишите с точкой.

## С нормализацией

In [36]:
kmeans = KMeans(n_clusters=3, n_init=1, random_state=42)
kmeans_mini_batch = MiniBatchKMeans(n_clusters=3, n_init=1, random_state=42)

In [37]:
kmeans.fit(X)
kmeans_mini_batch.fit(X)


C:\Users\vetak\anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:887: UserWarning: MiniBatchKMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can prevent it by setting batch_size >= 2048 or by setting the environment variable OMP_NUM_THREADS=1
  warnings.warn(


MiniBatchKMeans(n_clusters=3, n_init=1, random_state=42)

In [38]:
labels_k = kmeans.labels_
labels_mini_batch = kmeans_mini_batch.labels_

In [39]:
print('V_measure_kmean',round(v_measure_score(y, labels_k),1))
print('V_measure_mini_batch',round(v_measure_score(y, labels_mini_batch),1))

V_measure_kmean 0.8
V_measure_mini_batch 0.7


##  Без нормализации

In [40]:
n_samples = 1500
dataset = datasets.make_blobs(n_samples=n_samples, centers=2, center_box=(-7.0, 7.5),
                              cluster_std=[1.4, 1.7],
                              random_state=42)
X_2, _ = datasets.make_blobs(n_samples=n_samples, random_state=170, centers=[[-4, -3]], cluster_std=[1.9])
transformation = [[1.2, -0.8], [-0.4, 1.7]]
X_2 = np.dot(X_2, transformation)
X, y = np.concatenate((dataset[0], X_2)), np.concatenate((dataset[1], np.array([2] * len(X_2))))

In [41]:
kmeans = KMeans(n_clusters=3, n_init=1, random_state=42)
kmeans_mini_batch = MiniBatchKMeans(n_clusters=3, n_init=1, random_state=42)

kmeans.fit(X)
kmeans_mini_batch.fit(X)

labels_k = kmeans.labels_
labels_mini_batch = kmeans_mini_batch.labels_

print('V_measure_kmean',round(v_measure_score(y, labels_k),1))
print('V_measure_mini_batch',round(v_measure_score(y, labels_mini_batch),1))

V_measure_kmean 0.8
V_measure_mini_batch 0.8


C:\Users\vetak\anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:887: UserWarning: MiniBatchKMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can prevent it by setting batch_size >= 2048 or by setting the environment variable OMP_NUM_THREADS=1
  warnings.warn(


Задание 7.14.5
Рассмотрим агломеративную кластеризацию. Сравним, как влияет на качество кластеризации разный тип расстояния между кластерами.

Обучите на исходном датасете четыре модели AgglomerativeClustering с параметром n_clusters=3, меняя параметр linkage.

В качестве ответа укажите максимальное значение v-меры, полученное при помощи данных моделей. В задании может понадобиться, а может не понадобиться нормализация и это нужно проверить во время решения задания. Ответ округлите до десятых и запишите с точкой.


## Без нормализации

In [42]:
dict_linkage = dict()
list_linkage = ['ward', 'complete', 'average', 'single']

for link in list_linkage:
    model = AgglomerativeClustering(n_clusters=3, linkage=link)
    model.fit(X)
    labels = model.labels_
    dict_linkage.update({link: round(v_measure_score(y, labels),1)})
    
dict_linkage

{'ward': 0.7, 'complete': 0.4, 'average': 0.5, 'single': 0.0}

## С нормализацей

In [43]:
X = StandardScaler().fit_transform(X)

dict_linkage = dict()
list_linkage = ['ward', 'complete', 'average', 'single']

for link in list_linkage:
    model = AgglomerativeClustering(n_clusters=3, linkage=link)
    model.fit(X)
    labels = model.labels_
    dict_linkage.update({link: round(v_measure_score(y, labels),1)})
    
dict_linkage

{'ward': 0.9, 'complete': 0.6, 'average': 0.7, 'single': 0.0}

Задание 7.14.6
Сравним, как влияет предварительный расчёт матрицы смежности на качество агломеративной кластеризации.

Обучите на исходном датасете две модели AgglomerativeClustering:

с параметром n_clusters=3
с параметром n_clusters=3 и предварительно посчитанной матрицей смежности для объектов датасета
Построить матрицу смежности можно с помощью кода:

In [44]:
from sklearn.neighbors import kneighbors_graph

connectivity = kneighbors_graph(X, n_neighbors=
6, include_self=
False)
connectivity = 0.5 * (connectivity + connectivity.T)

In [45]:
agg_common = AgglomerativeClustering(n_clusters=3)
agg_connectivity = AgglomerativeClustering(n_clusters=3, connectivity=connectivity)

agg_common.fit(X)
agg_connectivity.fit(X)

labels_agg_common = agg_common.labels_
labels_agg_connectivity = agg_connectivity.labels_

print('V_measure Agglometative_common',round(v_measure_score(y, labels_agg_common),2))
print('V_measure_Agglomerative_connectivity',round(v_measure_score(y, labels_agg_connectivity),2))

V_measure Agglometative_common 0.91
V_measure_Agglomerative_connectivity 0.88


Задание 7.14.7
Алгоритм DBSCAN очень требователен к параметрам: небольшое изменение в параметре eps или min_samples может изменить результат и качество кластеризации.

Обучите на исходном датасете две модели DBSCAN:

с параметрами eps=0.9, min_samples=35
с параметрами eps=0.8, min_samples=35
В качестве ответа укажите максимальное значение v-меры, полученное при помощи данных моделей. В задании может понадобиться, а может не понадобиться нормализация и это нужно проверить во время решения задания. Ответ округлите до сотых и запишите с точкой.

## Без нормализации 


In [46]:

n_samples = 1500
dataset = datasets.make_blobs(n_samples=n_samples, centers=2, center_box=(-7.0, 7.5),
                              cluster_std=[1.4, 1.7],
                              random_state=42)
X_2, _ = datasets.make_blobs(n_samples=n_samples, random_state=170, centers=[[-4, -3]], cluster_std=[1.9])
transformation = [[1.2, -0.8], [-0.4, 1.7]]
X_2 = np.dot(X_2, transformation)
X, y = np.concatenate((dataset[0], X_2)), np.concatenate((dataset[1], np.array([2] * len(X_2))))

In [47]:
dbscan_0 = DBSCAN(eps=0.9, min_samples=35)
dbscan_1 = DBSCAN(eps=0.8, min_samples=35)

dbscan_0.fit(X)
dbscan_1.fit(X)

labels_0 = dbscan_0.labels_
labels_1 = dbscan_1.labels_

print('V_measure DBSCAN 0.9',round(v_measure_score(y, labels_0),2))
print('V_measure DBSCAN 0.8',round(v_measure_score(y, labels_1),2))

V_measure DBSCAN 0.9 0.77
V_measure DBSCAN 0.8 0.71


# С Нормализацией

In [48]:
X = StandardScaler().fit_transform(X)

dbscan_0 = DBSCAN(eps=0.9, min_samples=35)
dbscan_1 = DBSCAN(eps=0.8, min_samples=35)

dbscan_0.fit(X)
dbscan_1.fit(X)

labels_0 = dbscan_0.labels_
labels_1 = dbscan_1.labels_

print('V_measure DBSCAN 0.9',round(v_measure_score(y, labels_0),2))
print('V_measure DBSCAN 0.8',round(v_measure_score(y, labels_1),2))

V_measure DBSCAN 0.9 0.0
V_measure DBSCAN 0.8 0.0


# ВЫВОД НА DBSCAN НОРМАЛИЗАЦИЯ НЕ НУЖНА

Задание 7.14.8

Особенностью алгоритма DBSCAN является то, что помимо кластеризации этот алгоритм определяет выбросы в выборке. Посмотрим на качество кластеризации без учёта таких объектов.

Обучите на исходном датасете модель DBSCAN с параметрами eps=0.9, min_samples=35. Посчитайте значение v-меры только для основных и граничных объектов выборки, то есть для объектов, что не являются выбросами. Ответ округлите до сотых и запишите с точкой.

# Без нормализации

In [82]:
X_norm = StandardScaler().fit_transform(X)
X, y = np.concatenate((dataset[0], X_2)), np.concatenate((dataset[1], np.array([2] * len(X_2))))

In [83]:
dbscan = DBSCAN(eps=0.9, min_samples=35)
dbscan.fit(X)
labels_dbscan = dbscan.labels_

In [84]:
not_noise_ind = np.where(labels_dbscan != -1)

In [85]:
print('V_measure DBSCAN без выбросов', round((v_measure_score(y[not_noise_ind], labels_dbscan[not_noise_ind])),2))

V_measure DBSCAN без выбросов 0.97


Задание 7.14.9
В курсе мы рассмотрели две метода нормализации данных:

MinMax нормализация — приведение данных к масштабу между  и .
Стандартная нормализация — данные имеют среднее  и стандартное отклонение .
Проверим, влияет ли предобработка данных на результат кластеризации. Обучите две модели AgglomerativeClustering с параметрами n_clusters=3:

на признаках исходного датасета,
предварительно трансформируйте признаки при помощи стандартной нормализации.
Посчитайте v-меру для получившихся результатов, в качестве ответа введите наибольшее значение. Ответ округлите до сотых и запишите с точкой.

In [92]:
agg_0 = AgglomerativeClustering(n_clusters=3) # без нормализации
agg_1 = AgglomerativeClustering(n_clusters=3) # с стандартной нормализацией

labels_0 = agg_0.fit_predict(X)
labels_1 = agg_1.fit_predict(X_norm)

print('V_measure Agglomerative common',round(v_measure_score(y, labels_0),2))
print('V_measure Agglomerative standart',round(v_measure_score(y, labels_1),2))

V_measure Agglomerative common 0.7
V_measure Agglomerative standart 0.91


Задание 7.14.10
Обучите две модели AgglomerativeClustering с параметрами n_clusters=3:

на признаках исходного датасета,
предварительно трансформируйте признаки при помощи MinMax нормализации.
Посчитайте v-меру для получившихся результатов, в качестве ответа введите наибольшее значение. Ответ округлите до сотых и запишите с точкой.

In [93]:
from sklearn.preprocessing import MinMaxScaler

In [95]:
X_minmax = MinMaxScaler().fit_transform(X)

In [97]:
agg_0 = AgglomerativeClustering(n_clusters=3) # без нормализации
agg_1 = AgglomerativeClustering(n_clusters=3) # с нормализацией MinMax

labels_0 = agg_0.fit_predict(X)
labels_1 = agg_1.fit_predict(X_minmax)

print('V_measure Agglomerative common',round(v_measure_score(y, labels_0),2))
print('V_measure Agglomerative minmax',round(v_measure_score(y, labels_1),2))


V_measure Agglomerative common 0.7
V_measure Agglomerative minmax 0.89
